In [1]:
import wikipedia as wp
import pandas as pd
import numpy as np

import json
from geopy.geocoders import Nominatim

import folium

print('Libraries imported')

Libraries imported


In [2]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

In [3]:
ndf = df[df['Borough']!='Not assigned']

In [4]:
g_df = pd.read_csv("Geospatial_Coordinates.csv")

In [5]:
G_df = pd.merge(ndf,g_df,on='Postal Code',how='inner')
G_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(G_df['Borough'].unique()),
        G_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [7]:
G_df.Borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

### Use geopy library to get the latitude and longitude values of Toronto.


In [8]:
address = '%Toronto, Toronto'

geolocator = Nominatim(user_agent="trnto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.65238435, -79.38356765.


#### Create a map of New York with neighborhoods superimposed on top.

In [9]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(G_df['Latitude'], G_df['Longitude'], G_df['Borough'], G_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [33]:
import json

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#### Define FourSquare Credentials and version

In [11]:
CLIENT_ID = 'X2XWA4MU4UUAFRKK52V4EEF342VTKQ1IWY05D5CLDAOYRNEM' # your Foursquare ID
CLIENT_SECRET = 'XG5POEYQT1GX5KOGV1WQICJBEZ24MCRMVR0UFC5BFPMBUXSR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X2XWA4MU4UUAFRKK52V4EEF342VTKQ1IWY05D5CLDAOYRNEM
CLIENT_SECRET:XG5POEYQT1GX5KOGV1WQICJBEZ24MCRMVR0UFC5BFPMBUXSR


Get the neighbourhood latitude and logitude values.

In [12]:
neighborhood_latitude = G_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = G_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = G_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [13]:
LIMIT = 100
radius = 50000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=X2XWA4MU4UUAFRKK52V4EEF342VTKQ1IWY05D5CLDAOYRNEM&client_secret=XG5POEYQT1GX5KOGV1WQICJBEZ24MCRMVR0UFC5BFPMBUXSR&v=20180605&ll=43.7532586,-79.3296565&radius=50000&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff14fb1d98e3a6ad12cdbb8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 238,
  'suggestedBounds': {'ne': {'lat': 44.203259050000455,
    'lng': -78.70783048492777},
   'sw': {'lat': 43.30325814999955, 'lng': -79.95148251507223}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.75984035203157,
  

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-16-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Edwards Gardens,Park,43.733574,-79.358329
2,Sunnybrook Park,Other Great Outdoors,43.722317,-79.355496
3,Naan & Kabob Halal,Middle Eastern Restaurant,43.742903,-79.305148
4,Kostas Meat Market,Greek Restaurant,43.760605,-79.301830


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
            # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=G_df['Neighbourhood'],
                                   latitudes=G_df['Latitude'],
                                   longitudes=G_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(10257, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,Parkwoods,43.753259,-79.329656,Naan & Kabob Halal,43.742903,-79.305148,Middle Eastern Restaurant


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,100,100,100,100,100,100
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Wilson Heights, Downsview North",100,100,100,100,100,100
Bayview Village,95,95,95,95,95,95
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
...,...,...,...,...,...,...
"Willowdale, Willowdale West",100,100,100,100,100,100
Woburn,99,99,99,99,99,99
Woodbine Heights,100,100,100,100,100,100


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 251 uniques categories.


In [23]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[162]] + list(toronto_onehot.columns[:162]) + list(toronto_onehot.columns[163:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,New American Restaurant,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(10257, 251)

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,New American Restaurant,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Arts & Crafts Store,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt,0.00,0.00,0.00,0.0,0.01,0.0,0.0,0.0,0.01,...,0.01,0.00,0.0,0.0,0.000000,0.00,0.010000,0.00,0.0,0.0
1,"Alderwood, Long Branch",0.01,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.01,...,0.01,0.02,0.0,0.0,0.020000,0.01,0.000000,0.02,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.00,0.00,0.01,0.0,0.01,0.0,0.0,0.0,0.00,...,0.00,0.01,0.0,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
3,Bayview Village,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.00,0.0,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.00,0.00,0.01,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.01,0.0,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Willowdale West",0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.00,0.0,0.0,0.010000,0.00,0.000000,0.00,0.0,0.0
95,Woburn,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.00,0.0,0.0,0.020202,0.00,0.010101,0.00,0.0,0.0
96,Woodbine Heights,0.00,0.01,0.00,0.0,0.02,0.0,0.0,0.0,0.00,...,0.00,0.01,0.0,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
97,York Mills West,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.00,0.0,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0


In [26]:
toronto_grouped.shape

(99, 251)

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.07
1           Coffee Shop  0.07
2  Caribbean Restaurant  0.05
3          Noodle House  0.04
4     Indian Restaurant  0.04


----Alderwood, Long Branch----
          venue  freq
0        Bakery  0.05
1   Coffee Shop  0.05
2          Café  0.04
3  Liquor Store  0.03
4  Burger Joint  0.03


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.06
1             Clothing Store  0.04
2                 Restaurant  0.04
3  Middle Eastern Restaurant  0.03
4          Korean Restaurant  0.03


----Bayview Village----
                       venue  freq
0                Coffee Shop  0.08
1                Supermarket  0.07
2  Middle Eastern Restaurant  0.04
3          Korean Restaurant  0.03
4        Japanese Restaurant  0.03


----Bedford Park, Lawrence Manor East----
           venue  freq
0    Coffee Shop  0.09
1           Café  0.08
2  Grocery Store  0.

                venue  freq
0         Coffee Shop  0.09
1                Café  0.07
2                Park  0.06
3  Italian Restaurant  0.06
4              Bakery  0.05


----Leaside----
                venue  freq
0                Park  0.11
1                Café  0.10
2  Italian Restaurant  0.06
3         Coffee Shop  0.05
4              Bakery  0.03


----Little Portugal, Trinity----
         venue  freq
0         Park  0.13
1         Café  0.11
2  Coffee Shop  0.04
3      Brewery  0.03
4  Pizza Place  0.03


----Malvern, Rouge----
            venue  freq
0     Zoo Exhibit  0.11
1     Coffee Shop  0.07
2        Pharmacy  0.07
3  Sandwich Place  0.05
4     Gas Station  0.05


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                  venue  freq
0    Chinese Restaurant  0.12
1                Bakery  0.07
2  Caribbean Restaurant  0.05
3       Bubble Tea Shop  0.05
4           Supermarket  0.04


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park So

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Noodle House,Indian Restaurant,Pizza Place,Sushi Restaurant,Sandwich Place,Park,Bakery
1,"Alderwood, Long Branch",Coffee Shop,Bakery,Café,Grocery Store,Seafood Restaurant,Liquor Store,Burger Joint,Burrito Place,Brewery,Sandwich Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Restaurant,Clothing Store,Middle Eastern Restaurant,Korean Restaurant,Grocery Store,Bakery,Café,French Restaurant,Toy / Game Store
3,Bayview Village,Coffee Shop,Supermarket,Middle Eastern Restaurant,Korean Restaurant,Shopping Mall,Japanese Restaurant,Bakery,Restaurant,Grocery Store,Caribbean Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Café,Park,Grocery Store,Bakery,Sushi Restaurant,Italian Restaurant,Liquor Store,Ice Cream Shop,Supermarket


In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 2, 1, 4, 1, 4, 1, 4])

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
G_df['Neighborhood'] = G_df['Neighbourhood']
toronto_merged = G_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,Parkwoods,2,Middle Eastern Restaurant,Grocery Store,Caribbean Restaurant,Bakery,Burger Joint,Japanese Restaurant,Café,Restaurant,Supermarket,Liquor Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village,2,Burger Joint,Supermarket,Park,Coffee Shop,Italian Restaurant,Grocery Store,Middle Eastern Restaurant,Liquor Store,Pet Store,Chinese Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"Regent Park, Harbourfront",4,Coffee Shop,Park,Café,Hotel,Plaza,Farmers Market,Gastropub,Japanese Restaurant,Italian Restaurant,Historic Site
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,"Lawrence Manor, Lawrence Heights",2,Coffee Shop,Italian Restaurant,Café,Bakery,Vietnamese Restaurant,Bank,Park,Hockey Arena,Fast Food Restaurant,Sandwich Place
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"Queen's Park, Ontario Provincial Government",4,Park,Café,Coffee Shop,Farmers Market,Bakery,Hotel,Sandwich Place,Plaza,Japanese Restaurant,Dance Studio


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,"Islington Avenue, Humber Valley Village",0,Coffee Shop,Sandwich Place,Grocery Store,Liquor Store,Pharmacy,Pizza Place,Restaurant,Café,Bakery,Italian Restaurant
18,Scarborough,"Guildwood, Morningside, West Hill",0,Coffee Shop,Pharmacy,Bank,Sandwich Place,Park,Restaurant,Indian Restaurant,Gas Station,Fast Food Restaurant,Breakfast Spot
22,Scarborough,Woburn,0,Coffee Shop,Park,Pharmacy,Restaurant,Caribbean Restaurant,Fast Food Restaurant,Hotel,Sandwich Place,Pizza Place,Breakfast Spot
26,Scarborough,Cedarbrae,0,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Caribbean Restaurant,Bank,Burger Joint,Fried Chicken Joint,Breakfast Spot
32,Scarborough,Scarborough Village,0,Coffee Shop,Burger Joint,Park,Ice Cream Shop,Indian Restaurant,Supermarket,Burrito Place,Fish & Chips Shop,Fried Chicken Joint,Sports Bar
50,North York,Humber Summit,0,Coffee Shop,Italian Restaurant,Steakhouse,Hotel,Burger Joint,Sandwich Place,Indian Restaurant,Pharmacy,Ice Cream Shop,Fast Food Restaurant
51,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",0,Coffee Shop,Park,Grocery Store,Gym,Sandwich Place,Pharmacy,Bank,Beer Store,Burger Joint,Japanese Restaurant
57,North York,"Humberlea, Emery",0,Pizza Place,Coffee Shop,Gas Station,Bank,Vietnamese Restaurant,Grocery Store,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Sandwich Place
70,Etobicoke,Westmount,0,Coffee Shop,Sandwich Place,Restaurant,Pizza Place,Bank,Bakery,Liquor Store,Golf Course,Gas Station,Beer Store
77,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",0,Coffee Shop,Restaurant,Hotel,Pharmacy,Sandwich Place,Chinese Restaurant,Bakery,Italian Restaurant,Bank,Golf Course


In [39]:
toronto_merged.iloc[:,[1,7,8,9,10,11,12,13,14,15,16]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Middle Eastern Restaurant,Grocery Store,Caribbean Restaurant,Bakery,Burger Joint,Japanese Restaurant,Café,Restaurant,Supermarket,Liquor Store
1,North York,Burger Joint,Supermarket,Park,Coffee Shop,Italian Restaurant,Grocery Store,Middle Eastern Restaurant,Liquor Store,Pet Store,Chinese Restaurant
2,Downtown Toronto,Coffee Shop,Park,Café,Hotel,Plaza,Farmers Market,Gastropub,Japanese Restaurant,Italian Restaurant,Historic Site
3,North York,Coffee Shop,Italian Restaurant,Café,Bakery,Vietnamese Restaurant,Bank,Park,Hockey Arena,Fast Food Restaurant,Sandwich Place
4,Downtown Toronto,Park,Café,Coffee Shop,Farmers Market,Bakery,Hotel,Sandwich Place,Plaza,Japanese Restaurant,Dance Studio
...,...,...,...,...,...,...,...,...,...,...,...
98,Etobicoke,Park,Coffee Shop,Ice Cream Shop,Café,Bakery,Brewery,Liquor Store,Grocery Store,Italian Restaurant,Gastropub
99,Downtown Toronto,Coffee Shop,Café,Park,Farmers Market,Historic Site,Grocery Store,Sandwich Place,Bakery,Bookstore,Japanese Restaurant
100,East Toronto,Park,Café,Coffee Shop,Beach,Brewery,Pizza Place,Gastropub,Bakery,Indian Restaurant,French Restaurant
101,Etobicoke,Park,Italian Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Ice Cream Shop,Gastropub,Scenic Lookout


In [42]:
toronto_venues[['Neighborhood','Venue Category']]
toronto_venues.groupyby('Venue Category')

AttributeError: 'DataFrame' object has no attribute 'groupyby'

In [43]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').count().reset_index()
toronto_grouped

,Neighborhood,New American Restaurant,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Arts & Crafts Store,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,"Alderwood, Long Branch",100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2,"Bathurst Manor, Wilson Heights, Downsview North",100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
3,Bayview Village,95,95,95,95,95,95,95,95,95,...,95,95,95,95,95,95,95,95,95,95
4,"Bedford Park, Lawrence Manor East",100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Willowdale West",100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
95,Woburn,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99
96,Woodbine Heights,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
97,York Mills West,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
